### Векторные представления слов (word embeddings)

Задача поиск похожих по эмбеддингам

Скачиваем датасет ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать 
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов

на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу.

3. Проверить насколько хорошо работают подходы


In [2]:
pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 646 kB 4.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391567 sha256=bea7a47e3829329b3322261dcbcc05bdc68060929bb3e405822d356f20ef365e
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [4]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 8.2 MB 8.2 MB/s 


In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import re
from html.parser import HTMLParser
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from string import punctuation
from pymystem3 import Mystem
from gensim.models import Word2Vec, FastText
import annoy
import tqdm
from pymorphy2 import MorphAnalyzer

In [6]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-05 07:48:49--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-05 07:48:49--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2b24829c405be60160455cf07f.dl.dropboxusercontent.com/cd/0/inline/BmmtnBag22SS8sXX6trym7HafUPGKCoLmwFd7gc3TxzZ8QkZuAIzU92vq1VOplbGwdqTNGWT-aEwWsrgqOG66Af43VrxkOsglaxrkmr0uUX3UklNMwhnmNm-HwqQZDngN4UQxR56bElfChW7Ekcgs8iXDgHA_lbMpS46fNS3EyaONg/file# [following]
--2022-06-05 07:48:50--  https://uc2b24829c405be60160455cf07f.dl.dropboxusercontent.com/cd/0/inline/BmmtnBag22SS8sXX6trym7HafUPGKCoLmwFd7gc3TxzZ8QkZuAIzU92vq1VOplbGwd

In [7]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = pd.concat([positive, negative], ignore_index=True)

In [8]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [26]:
df = df['text']

In [28]:
len(df)

226834

In [29]:
type(df[0])

str

In [9]:
import gensim.downloader as api

In [10]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [ ]:
import gensim.downloader as api

word_vectors = api.load('word2vec-ruscorpora-300')  # загрузим предтренированные вектора слов из gensim-data
# В модели "word2vec-ruscorpora-300" после слова указывается часть речи: NOUN (существительное), ADJ (прилагательное) и так далее.
# выведим слово наиболее близкое к 'woman', 'king' и далекое от 'man'
result = word_vectors.most_similar(positive=['женщина_NOUN', 'король_NOUN'], negative=['мужчина_NOUN'])
print("{}: {:.4f}".format(*result[0]))

королева_NOUN: 0.7314


In [11]:
#word_vectors.key_to_index

In [22]:
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=bab9a86e241d96bcf2e101276936b5f30c73f67239575fb383c7af88137717a0
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [23]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [24]:
!ls

negative.csv  positive.csv  sample_data


In [31]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [36]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for line in tqdm_notebook(df):
    spls = preprocess_txt(line)
    sentences.append(spls)
    c += 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/226834 [00:00<?, ?it/s]

In [37]:
sentences = [i for i in sentences if len(i) > 2]

In [38]:
sentences[0]

['firsttimee',
 'школотый',
 'поверь',
 'самый',
 'd',
 'общество',
 'профилировать',
 'предмет',
 'тип']

In [39]:
Word2Vec?

In [41]:
modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=10)

In [43]:
modelFT = FastText(sentences=sentences, size=300, min_count=10, window=5, workers=8)

In [45]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in tqdm_notebook(df):
    n_w2v = 0
    n_ft = 0
    index_map[counter] = line # помещаю твит в словарь index_map
    question = preprocess_txt(line) # преобразую твит в вектор w2v и FT
        
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    # упаковываю вектора в ANNOY
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
            
    counter += 1
# разбиваю векторные пространства на 10 кластеров
w2v_index.build(10)
ft_index.build(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/226834 [00:00<?, ?it/s]

True

In [49]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 6, )
    return [index_map[i] for i in answers[1:]]

In [80]:
for i in range(1, 6):
  print(f"Твит: {df[i]} \n")
  print(f"Похожие V2V:")
  for i in get_response(df[i], w2v_index, modelW2V, index_map):
    print(i)
  print(f"\n Похожие FT:")
  for i in get_response(TEXT, ft_index, modelFT, index_map):
    print(i)
  print(f"\n END \n")

Твит: Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D 

Похожие V2V:
Наши мальчики тоже гадать умеет:)
А особеено это хорошо получается у Миши:) http://t.co/g8mIiEfEXK
Говорил надолго вместе, навсегда, а оказывается пустые слова, больно и обидно:( 
 Все равно люблю его очень сильно;*
Случилось то,о чем мой мальчик мечтал.забрал мою Соню из сада)
Ну почему меня в гране всегда обслуживают какие-то сладенькие мальчики???(
Бедные парни.Ваши 15,16-ти летние девочки сами не знают чего хотят,мечась от одного к другому,набивают себе цену.В какой раз столкнулся:(...

 Похожие FT:
Мне жалко Джуна.Что такого случилось,что он сказал,что не дурак :(
@Mparkhimchik  ахахахха мило) 
я наверное была похожа на унылое  говно
@mazayk я еще раз убедилась, что ты ооооочень плохой ребенок)):*
@LalkaVerka @asutka999 я на себя реально не наговариваю(( вот это правда, вы красивые, а я правда некрасивая((( в классе я изгой и уродина(
@DToropova Я ДУМАЮ ЧТО ТЫ УЖЕ БОЛЬШАЯ ДЕВОЧКА,  САМА ВС